Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [2]:
import numpy as np

In [3]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
raw_train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
raw_test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

- Data wrangling and cleaning

In [4]:
def wrangle(X):
    """Wrangle (setup) train, validated, and test sets"""
    
    X = X.copy()
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    cols_with_zeroes = ['longitude', 'latitude', 'construction_year',
                       'gps_height', 'population']
    # storing the missing values in another column in case that's useful.
    # impute the missing values later.
    for col in cols_with_zeroes:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
    
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # unusuable variance or otherwise seemingly unfit data
    #   dropping the other columns didn't seem to do much unfortuntaley
    unusable_variance = ['recorded_by', 'id', 'amount_tsh', 'public_meeting', 'scheme_name', 'subvillage', 'ward',
                        'num_private', 'lga']
    X = X.drop(columns=unusable_variance)
        
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

In [5]:
train = wrangle(raw_train)
test  = wrangle(raw_test)

- Setup other variables

In [6]:
target = 'status_group'
train_features = train.drop(columns=[target])
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 50].index.tolist()
features = numeric_features + categorical_features

X_train = train[features]
y_train = train[target]

- [X] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.

In [7]:
import category_encoders as ce
# pipeline doesn't seem to handle categorical variables in the y_train column,
# or maybe I was'nt using a categorical function . target encdoer is probably doing nothen
encoder = ce.OrdinalEncoder()
encoder.fit(X_train)
encoder.transform(X_train).dtypes

encoder2 = ce.OrdinalEncoder()
encoder2.fit(y_train)
encoder2.transform(y_train).dtypes
X_train = encoder.transform(X_train)
y_train = encoder2.transform(y_train)

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import LogisticRegressionCV, RidgeCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier # should use classifier for this problem

pipeline = make_pipeline(
    SimpleImputer(),
    StandardScaler(),
    SelectKBest(f_regression),
    # also tried elasticnetcv 
    LogisticRegressionCV()
)

pipeline_tree = make_pipeline(
    ce.TargetEncoder(min_samples_leaf=1, smoothing=1), 
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=50, n_jobs=-1)#RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42)
)

param_distributions = {
    'simpleimputer__strategy' : ['mean', 'median'],
    'logisticregressioncv__Cs' : np.arange(8, 12, 1),
    #'logisticregressioncv__penalty' : ['l2', 'elasticnet'] # says elasticnet is unsuppored wonder if version mismatch
}

pd_tree = {
    'targetencoder__min_samples_leaf' : [1, 2],
    'targetencoder__smoothing' : [0.85, 1, 1.15],
    'simpleimputer__strategy' : ['mean', 'median'],
}

In [9]:
 search  = RandomizedSearchCV(
        pipeline,
        param_distributions=param_distributions,
        n_iter=30,
        cv=5,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=10,
        return_train_score=True,
    )

y_train
search.fit(X_train, y_train)

c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  22 out of  40 | elapsed:   18.8s remaining:   15.3s
[Parallel(n_jobs=-1)]: Done  27 out of  40 | elapsed:   25.4s remaining:   12.2s
[Parallel(n_jobs=-1)]: Done  32 out of  40 | elapsed:   27.1s remaining:    6.7s
[Parallel(n_jobs=-1)]: Done  37 out of  40 | elapsed:   30.2s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   30.7s finished
c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarn

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,
                                                            missing_values=nan,
                                                            strategy='mean',
                                                            verbose=0)),
                                             ('standardscaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('selectkbest',
                 

In [10]:
# kinda bad mae
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'median', 'logisticregressioncv__Cs': 8}
Cross-validation MAE 0.41723905723905724


In [11]:
 search2  = RandomizedSearchCV(
        pipeline_tree,
        param_distributions=pd_tree,
        n_iter=30,
        cv=5,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=10,
        return_train_score=True,
    )
search2.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done  44 out of  60 | elapsed:   42.5s remaining:   15.4s
[Parallel(n_jobs=-1)]: Done  51 out of  60 | elapsed:   44.5s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done  58 out of  60 | elapsed:   49.6s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   49.7s fi

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('targetencoder',
                                              TargetEncoder(cols=None,
                                                            drop_invariant=False,
                                                            handle_missing='value',
                                                            handle_unknown='value',
                                                            min_samples_leaf=1,
                                                            return_df=True,
                                                            smoothing=1.0,
                                                            verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                           

In [12]:
# Mean absolute error off by around 0.24
print('Best hyperparameters', search2.best_params_)
print('Cross-validation MAE', -search2.best_score_)

Best hyperparameters {'targetencoder__smoothing': 1.15, 'targetencoder__min_samples_leaf': 1, 'simpleimputer__strategy': 'mean'}
Cross-validation MAE 0.24904040404040403


In [13]:
from sklearn.model_selection import cross_val_score

# pipeline as decided by the algorithm
final_pipeline = make_pipeline(
    ce.TargetEncoder(smoothing=0.85, min_samples_leaf=1),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=50, n_jobs=-1)
)

cross_val_score(final_pipeline, X_train, y_train, scoring='neg_mean_absolute_error')

c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py:356: DataConversionW

array([-0.25520202, -0.25141414, -0.25686869])

In [14]:
# pipeline from the previous assignment, that is. Want to compore the cross val scores. 
previous_pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(n_estimators=50, n_jobs=-1)
)

cross_val_score(previous_pipeline, X_train, y_train, scoring='neg_mean_absolute_error')

c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
c:\users\school\appdata\local\programs\python\python37\lib\site-packages\sklearn\pipeline.py:356: DataConversionW

array([-0.25318182, -0.24994949, -0.255     ])

New score seems very silghtly better, but not too much. Thin I'll just keep my old score.

- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

- [X] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.